# MIDS - w261 Machine Learning At Scale


## Project 5


---
__Name:__  Hyera Moon   
__Week:__   5




# Table of Contents <a name="TOC"></a> 

1.  [HW Intructions](#1)   
2.  [HW References](#2)
3.  [HW Problems](#3)   
1.  [HW Introduction](#1)   
2.  [HW References](#2)
3.  [HW  Problems](#3)   
    5.0.  [HW5.0](#5.0)   
    5.1.  [HW5.1](#5.1)   
    5.2.  [HW5.2](#5.2)   
    5.3.  [HW5.3](#5.3)    
    
   

<a name="1"></a>
# 1 Instructions
[Back to Table of Contents](#TOC)


  
<a name="2">
# 2 Useful References
[Back to Table of Contents](#TOC)

* See async and live lectures for this week

<a name="3">
# HW Problems
[Back to Table of Contents](#TOC)

## 3.  HW5.0  data warehouse; star schema<a name="5.0"></a>
[Back to Table of Contents](#TOC)

- What is a data warehouse? What is a Star schema? When is it used?

A data warehouse is a central repository of integrated data from a wide range of sources within a company/enterprise. It stores current and historical data and forms a foundation for business intelligence (e.g. create analytical reports) and data science. Typically it stores relational data but increasingly semi-structured data and unstructured data are also being stored in the data warehouse.

In data warehousing, a Star schema is a simple style of data mart schema: it organizes the data into facts and dimensions and optimizes querying of large datsets. The star schema consists of one or more fact tables referencing any number of dimensions table. In fact, the fact table holds foreign keys which are the primary key of each associated dimension table. Thus, each fact is surrounded with its associated dimensions and this results in a star shape looking diagram.
Also the fact table contains one numeric measure thus typically, there will be one fact table and star schema for each measure table.

## 3.  HW5.1 Databases: 3NF; denormalized <a name="5.1"></a>
[Back to Table of Contents](#TOC)

- In the database world What is 3NF? Does machine learning use data in 3NF? If so why? 
- In what form does ML consume data?
- Why would one use log files that are denormalized?

3NF stands for third normal form and it is the third step of a database normalization process designed to reduce the duplication of data. It is designed to improve database processing while minimizing storage costs and thus, it is used for OLTP since its applications are characterized by a high volume of small transactions (large storage needed) in real time (fast processing needed).

Machine learning does not use data in 3NF but uses denormalized data instead. Unlike normalized data, denormalized data is typically used for OLAP and data science to get the full picture (e.g. extract historical data that has accumulated over a long period of time). In a normalized model, the related data are stored in separate tables to remove any redundant data and to group those related data will require a join operation. In constrast, the denormalization process add these redundant data where it will help the most via adding extra attributes or new tables. In other words, it will try to improve the read performance of a database by adding redundant copies of data or by grouping data. Machine learning which needs to carry out large numbers of read operations, needs the joined data, that is, the denormalized data.

We would use log files that are denormalized when we need to maintain history of changes in the log files, for example in OLAP. This can be done by denormalization process by adding a table containing the history of these changes. This will help to query the log files more easily and faster than with normalized files.

## 3.  HW5.2  Memory-backed map-side<a name="5.2"></a>
[Back to Table of Contents](#TOC)

Using MRJob, implement a hashside join (memory-backed map-side) for left, right and inner joins. Use the following tables for this HW and join based on the country code (third column of the transactions table and the second column of the Countries table:

<PRE>
transactions.dat
Alice Bob|$10|US
Sam Sneed|$1|CA
Jon Sneed|$20|CA
Arnold Wesise|$400|UK
Henry Bob|$2|US
Yo Yo Ma|$2|CA
Jon York|$44|CA
Alex Ball|$5|UK
Jim Davis|$66|JA

Countries.dat
United States|US
Canada|CA
United Kingdom|UK
Italy|IT

</PRE>

Justify which table you chose as the Left table in this hashside join.

Please report the number of rows resulting from:

- (1) Left joining Table Left with Table Right
- (2) Right joining Table Left with Table Right
- (3) Inner joining Table Left with Table Right


The smaller table Countries.dat should be the one to be loaded into memory (in memory hash) to fit into memory and broadcasted to all nodes and mappers. The left table should be the table that will be loaded into memory, in this case, the smaller table Countries.dat

In [1]:
%%writefile transactions.dat
Alice Bob|$10|US
Sam Sneed|$1|CA
Jon Sneed|$20|CA
Arnold Wesise|$400|UK
Henry Bob|$2|US
Yo Yo Ma|$2|CA
Jon York|$44|CA
Alex Ball|$5|UK
Jim Davis|$66|JA

Writing transactions.dat


In [2]:
%%writefile Countries.dat
United States|US
Canada|CA
United Kingdom|UK
Italy|IT

Writing Countries.dat


In [50]:
%%writefile rightjoin.py

from mrjob.job import MRJob
from mrjob.job import MRStep

class rightjoin(MRJob):
    ''' Assuming left table is Countries.dat(in-memory) and right table is transactions.dat
    '''
    
    def steps(self):
        return[
            MRStep(mapper_init = self.mapper_init,
                   mapper = self.mapper)
        ]

    
    def mapper_init(self):
        self.countries ={}  # dictionary with key = country code and value = country name
        with open("Countries.dat", "r") as countryfile:
            for line in countryfile:
                line = line.strip()
                name, code = line.split('|')
                self.countries[code] = name

        
    def mapper(self, _, line):
        customer, amount, countryID  = line.split('|')
        countryname = self.countries.get(countryID, "None")
        yield customer, (amount, countryID, countryname)
        

if __name__ == '__main__':
    rightjoin.run()
        
    

Overwriting rightjoin.py


In [51]:
!chmod a+x rightjoin.py

In [56]:
%reload_ext autoreload
%autoreload 2

from rightjoin import rightjoin
mr_job = rightjoin(args=['transactions.dat','--file=Countries.dat'])

with mr_job.make_runner() as runner:
    runner.run()
    count = 0
    print "Right join (assuming Countries.dat is the left table and transactions.dat is the right table)"
    for line in runner.stream_output():
        key, value = mr_job.parse_output_line(line)
        print "{0:>15} |{1:>15} |{2:>15} |{3:>15}".format(key, value[0], value[1], value[2])
        count = count + 1
        
print ""
print "There are %s rows" %(count)

Right join (assuming Countries.dat is the left table and transactions.dat is the right table)
      Alice Bob |            $10 |             US |  United States
      Sam Sneed |             $1 |             CA |         Canada
      Jon Sneed |            $20 |             CA |         Canada
  Arnold Wesise |           $400 |             UK | United Kingdom
      Henry Bob |             $2 |             US |  United States
       Yo Yo Ma |             $2 |             CA |         Canada
       Jon York |            $44 |             CA |         Canada
      Alex Ball |             $5 |             UK | United Kingdom
      Jim Davis |            $66 |             JA |           None

There are 9 rows


In [53]:
%%writefile innerjoin.py

from mrjob.job import MRJob
from mrjob.job import MRStep

class innerjoin(MRJob):
    ''' Assuming left table is Countries.dat(in-memory) and right table is transactions.dat
    '''
    
    def steps(self):
        return[
            MRStep(mapper_init = self.mapper_init,
                   mapper = self.mapper)
        ]

    
    def mapper_init(self):
        self.countries ={}  # dictionary with key = country code and value = country name
        with open("Countries.dat", "r") as countryfile:
            for line in countryfile:
                line = line.strip()
                name, code = line.split('|')
                self.countries[code] = name

        
    def mapper(self, _, line):
        customer, amount, countryID  = line.split('|')
        countryname = self.countries.get(countryID, "None")
        if countryname != "None":
            yield customer, (amount, countryID, countryname)

        
if __name__ == '__main__':
    innerjoin.run()

Overwriting innerjoin.py


In [54]:
!chmod a+x innerjoin.py

In [55]:
%reload_ext autoreload
%autoreload 2

from innerjoin import innerjoin
mr_job = innerjoin(args=['transactions.dat','--file=Countries.dat'])

with mr_job.make_runner() as runner:
    runner.run()
    count = 0
    print "Inner join"
    for line in runner.stream_output():
        key, value = mr_job.parse_output_line(line)
        print "{0:>15} |{1:>15} |{2:>15} |{3:>15}".format(key, value[0], value[1], value[2])
        count = count + 1
        
print ""
print "There are %s rows" %(count)

Inner join
      Alice Bob |            $10 |             US |  United States
      Sam Sneed |             $1 |             CA |         Canada
      Jon Sneed |            $20 |             CA |         Canada
  Arnold Wesise |           $400 |             UK | United Kingdom
      Henry Bob |             $2 |             US |  United States
       Yo Yo Ma |             $2 |             CA |         Canada
       Jon York |            $44 |             CA |         Canada
      Alex Ball |             $5 |             UK | United Kingdom

There are 8 rows


In [288]:
%%writefile leftjoin.py

from mrjob.job import MRJob
from mrjob.job import MRStep

class leftjoin(MRJob):
    ''' Assuming left table is Countries.dat(in-memory) and right table is transactions.dat
    '''

    def steps(self):
        return[
            MRStep(mapper_init = self.mapper_init,
                   mapper = self.mapper,
                   reducer = self.reducer
                  )
        ]

    
    def mapper_init(self):
        self.countries ={}  # dictionary with key = country code and value = country name
        with open("Countries.dat", "r") as countryfile:
            for line in countryfile:
                line = line.strip()
                name, code = line.split('|')
                self.countries[code] = name
        
    def mapper(self, _, line):
        customer, amount, countryID  = line.split('|')
        for code in self.countries.keys():
            if code == countryID:
                yield code, (self.countries[code], customer, amount)
            else:
                yield code, (self.countries[code], "None", "None")
                
    
    def reducer(self, key, values):
        code = key  # all values grouped by country code
        customer_in_countriesFile = False
        for value in values:
            if value[1] != "None":
                customer_in_countriesFile = True
                yield value[1], (value[2], code, value[0])
        if customer_in_countriesFile == False:
            yield "None", ("None", code, value[0])
    
   

if __name__ == '__main__':
    leftjoin.run()

Overwriting leftjoin.py


In [ ]:
!chmod a+x leftjoin.py

In [289]:
%reload_ext autoreload
%autoreload 2

from leftjoin import leftjoin
mr_job = leftjoin(args=['transactions.dat','--file=Countries.dat'])
#mr_job = leftjoin(args=['transactions.dat','Countries.dat'])

with mr_job.make_runner() as runner:
    runner.run()
    count = 0
    print "Left join (assuming Countries.dat is the left table and transactions.dat is the right table)"
    for line in runner.stream_output():
        key, value = mr_job.parse_output_line(line)
        print "{0:>15} |{1:>15} |{2:>15} |{3:>15}".format(key, value[0], value[1], value[2])
        count = count + 1
        
print ""
print "There are %s rows" %(count)

Left join (assuming Countries.dat is the left table and transactions.dat is the right table)
      Jon Sneed |            $20 |             CA |         Canada
       Jon York |            $44 |             CA |         Canada
      Sam Sneed |             $1 |             CA |         Canada
       Yo Yo Ma |             $2 |             CA |         Canada
           None |           None |             IT |          Italy
      Alex Ball |             $5 |             UK | United Kingdom
  Arnold Wesise |           $400 |             UK | United Kingdom
      Alice Bob |            $10 |             US |  United States
      Henry Bob |             $2 |             US |  United States

There are 9 rows


## 3.  HW5.2.1 (OPTIONAL) Almost stateless reducer-side join  <a name="5.2.1"></a>
[Back to Table of Contents](#TOC)

The following MRJob code, implements a reduce-side join for an inner join. The reducer is almost stateless, i.e., uses as little memory as possible. Use the tables from HW5.2 for this HW and join based on the country code (third column of the transactions table and the second column of the Countries table perform. Perform  an left, right, inner joins using the code provided below and report the number of rows resulting from:

- (1) Left joining Table Left with Table Right
- (2) Right joining Table Left with Table Right
- (3) Inner joining Table Left with Table Right

Again make smart decisions about which table should be the left table (i.e., crosscheck the code). 

__Some notes on the code__ 
Here, the mapper receives its set of input splits either from the transaction table or from the countries table and makes the appropriate transformations: splitting the line into fields, and emitting a key/value. The key is the join key - in this case, the country code field of both sets of records. The mapper knows which file and type of record it is receiving based on the length of the fields. The records it emits contain the join field as the key, which acts as the partitioning key; We use the SORT_VALUES option, which ensures the values are sorted as well. Then, we employ a trick to ensure that for each join key, country records are seen always before transaction records. We achieve this by adding an arbitrary key to the front of the value: 'A' for countries, 'B' for customers. This makes countries sort before customers for each and every join/partition key. After that trick, the join is simply a matter of storing countries ('A' records) and crossing this array with each customer record.

In [ ]:
import sys, os, re
from mrjob.job import MRJob

class MRJoin(MRJob):

  # Performs secondary sort
  SORT_VALUES = True

  def mapper(self, _, line):
    splits = line.rstrip("\n").split("|")

    if len(splits) == 2: # country data
      symbol = 'A' # make country sort before transaction data
      country2digit = splits[1]
      yield country2digit, [symbol, splits]
    else: # person data
      symbol = 'B'
      country2digit = splits[2]
      yield country2digit, [symbol, splits]

  def reducer(self, key, values):
    countries = [] # should come first, as they are sorted on artificia key 'A'
    for value in values:
      if value[0] == 'A':
        countries.append(value)
      if value[0] == 'B':
        for country in countries:
          yield key, country[1:] + value[1:]

if __name__ == '__main__':
  MRJoin.run()

# 5.3 Pairwise similarity  - PHASE 1 <a name="5.3"></a>

In this part of the assignment we will focus on developing methods for detecting synonyms, using the Google 5-grams dataset. To accomplish this you must script two main tasks using MRJob:


#### (1) Using the systems tests data sets, write mrjob code to build the stripes
#### (2) Write mrjob code to build an inverted index from the stripes
#### (3) Using two (symmetric) comparison methods of your choice (e.g., correlations, distances, similarities), pairwise compare all stripes (vectors), and output to a file.   

__==Design notes for (1)== __  
For this task you will be able to modify the pattern we used in HW 3.2 (feel free to use the solution as reference). To total the word counts across the n-grams, output the support from the mappers using the total order inversion pattern:

<*word,count>   

to ensure that the support arrives before the cooccurrences.   

In addition to ensuring the determination of the total word counts, the mapper must also output co-occurrence counts for the pairs of words inside of each n-gram. Treat these words as a basket, as we have in HW 3, but count all stripes or pairs in both orders, i.e., count both orderings: (word1,word2), and (word2,word1), to preserve
symmetry in our output for (2).

__==Design notes for (3)==__   
For this task you will have to determine a method of comparison.
Here are a few that you might consider:

 - Jaccard
 - Cosine similarity
 - Spearman correlation
 - Euclidean distance
 - Taxicab (Manhattan) distance
 - Shortest path graph distance (a graph, because our data is symmetric!)
 - Pearson correlation
 - Kendall correlation
 ...

However, be cautioned that some comparison methods are more difficult to parallelize than others, and do not perform more associations than is necessary, since your choice of association will be symmetric.

Please use the inverted index (discussed in live session #5) based pattern to compute the pairwise (term-by-term) similarity matrix. 

In [231]:
%%writefile buildStripes.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import re
import mrjob
import json
import itertools
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.job import MRStep

class MRbuildStripes(MRJob):
  
  #START SUDENT CODE531_STRIPES
    def steps(self):
        return[
            MRStep(
                mapper_init = self.mapper_init,
                mapper = self.mapper,
                reducer = self.reducer
            )
        ]
    
    def mapper_init(self):
        self.stripes = {}
        
    def mapper(self, _, line):
        line = line.strip().lower()
        ngram, count, other = line.split('\t',2)
        count = int(count)
        words = ngram.split(' ')
        self.stripes = {}
                
        for subset in itertools.permutations(sorted(words), 2):  #instead of combinations to keep symmetry
            if subset[0] == subset[1]:  # in case two same words in a line
                continue
            elif subset[0] not in self.stripes.keys():
                self.stripes[subset[0]] = {}
                self.stripes[subset[0]][subset[1]] = count
            elif subset[1] not in self.stripes[subset[0]]:
                self.stripes[subset[0]][subset[1]] = count
            else:
                self.stripes[subset[0]][subset[1]] += count

        for key in self.stripes.keys():
            yield key, self.stripes[key]
            
        
    def reducer(self, key, values):
        temp_stripes = {}
        for value in values:
            for word, count in value.items():
                temp_stripes[word] = temp_stripes.get(word,0) + count
        yield key, temp_stripes
  

  #END SUDENT CODE531_STRIPES   
    
if __name__ == '__main__':
    MRbuildStripes.run()

Overwriting buildStripes.py


In [68]:
!chmod a+x buildStripes.py

In [187]:
%%writefile invertedIndex.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-


from __future__ import division
import collections
import re
import json
import math
#import numpy as np commented since not supported by Hadoop Python version
import itertools
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.protocol import JSONProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep

class MRinvertedIndex(MRJob):
    INPUT_PROTOCOL = JSONProtocol

  #START SUDENT CODE531_INV_INDEX
    def steps(self):
        return[
            MRStep(
                mapper=self.mapper,
                reducer=self.reducer
            )
        ]
    
    def mapper(self, key_word, stripes):
        words = stripes.keys()
        _len = len(words)
        for word in words:
        # Store the length of the document to use with JACCARD (|A| + |B|)
            yield word, (key_word, _len)
        
    def reducer(self, word, values):        
        d = collections.defaultdict(list)
        for value in values:
            d[word].append(value)
        yield word,d[word]

  #END SUDENT CODE531_INV_INDEX
        
if __name__ == '__main__':
    MRinvertedIndex.run() 

Overwriting invertedIndex.py


In [182]:
!chmod a+x invertedIndex.py

In [245]:
%%writefile similarity.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import collections
import re
import json
import math
#import numpy as np  commented since not supported by Hadoop Python version
import itertools
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep

class MRsimilarity(MRJob):
  
  #START SUDENT CODE531_SIMILARITY
    MRJob.SORT_VALUES
    
    def steps(self):
        JOBCONF_STEP1 = {}
        JOBCONF_STEP2 = { 
                # Must use -r hadoop mode for this sorting to work #
                'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                'mapreduce.partition.keycomparator.options':'-k1,1nr',
        }

        return [
            MRStep(jobconf=JOBCONF_STEP1,
                   mapper = self.mapper_pair_sim,
                   reducer = self.reducer_pair_sim
                  ),
            MRStep(jobconf=JOBCONF_STEP2,
                  mapper=None,
                  reducer=self.reducer_sort)
        ]
        
    def mapper_pair_sim(self, _, line):
        line = line.strip()
        index,posting = line.split("\t")
        posting = json.loads(posting)
        
        X = map(lambda x: x[0]+"."+str(x[1]) , posting)
        
        # taking advantage of symetry, output only (a,b), but not (b,a)
        for subset in itertools.combinations(sorted(set(X)), 2):
            yield subset[0]+"."+subset[1], 1
        
            
    def reducer_pair_sim(self,key,values):
        word1, word1_len, word2, word2_len = key.split(".")
        t =sum(values)
       
        cosine = (1.0/(float(word1_len)**0.5) * 1.0/(float(word2_len)**0.5))*t
        jaccard = t / ( int(word1_len) + int(word2_len) - t )
        overlap = t / min(int(word1_len), int(word2_len))
        dice = 2 * t / ( int(word1_len) + int(word2_len))
        avg = (jaccard + cosine + overlap + dice) / 4
        
        # rounding to 6 decimals
        avg = round(avg, 6)
        cosine = round(cosine, 6)
        jaccard = round(jaccard, 6)
        overlap = round(overlap, 6)
        dice = round(dice, 6)
        
        yield avg, (word1+" - "+word2, cosine, jaccard, overlap, dice)
    
    
    def reducer_sort(self,key,values):
        for value in values:
            yield key, value

  #END SUDENT CODE531_SIMILARITY
  
if __name__ == '__main__':
    MRsimilarity.run()

Overwriting similarity.py


In [198]:
!chmod a+x similarity.py

## HW5.3.1   Run Systems tests locally on small datasets (PHASE1) <a name="5.3.1"></a>  
[Back to Table of Contents](#TOC)

Complete 5.3 and systems test using the below test datasets. Phase 2 will focus on the entire Ngram dataset.

To help you through these tasks please verify that your code gives the results below (for stripes, inverted index, and pairwise similarities).

Test datasets:

* googlebooks-eng-all-5gram-20090715-0-filtered.txt [see below]
* atlas-boon-test [see below]
* stripe-docs-test [see below]


A large subset of the Google n-grams dataset

https://aws.amazon.com/datasets/google-books-ngrams/

which we have placed in a bucket/folder on Dropbox and on s3:

https://www.dropbox.com/sh/tmqpc4o0xswhkvz/AACUifrl6wrMrlK6a3X3lZ9Ea?dl=0 

s3://filtered-5grams/

In particular, this bucket contains (~200) files (10Meg each) in the format:

	(ngram) \t (count) \t (pages_count) \t (books_count)

The next cell shows the first 10 lines of the googlebooks-eng-all-5gram-20090715-0-filtered.txt file.


__DISCLAIMER__: Each record is already a 5-gram. In real life, we would calculate the stripes cooccurrence data from the raw text by windowing over the raw text and not from the 5-gram preprocessed data (as we are doing here).  Calculatating pairs on this 5-gram is a little corrupt as we will be double counting cooccurences. Having said that this exercise can still pull out some simialr terms. 

#### 1: unit/systems first-10-lines

In [65]:
%%writefile googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt
A BILL FOR ESTABLISHING RELIGIOUS	59	59	54
A Biography of General George	92	90	74
A Case Study in Government	102	102	78
A Case Study of Female	447	447	327
A Case Study of Limited	55	55	43
A Child's Christmas in Wales	1099	1061	866
A Circumstantial Narrative of the	62	62	50
A City by the Sea	62	60	49
A Collection of Fairy Tales	123	117	80
A Collection of Forms of	116	103	82

Writing googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt


#### 2: unit/systems atlas-boon

In [66]:
%%writefile atlas-boon-systems-test.txt
atlas boon	50	50	50
boon cava dipped	10	10	10
atlas dipped	15	15	15

Writing atlas-boon-systems-test.txt


#### 3: unit/systems stripe-docs-test

Three terms, A,B,C and their corresponding stripe-docs of co-occurring terms

- DocA {X:20, Y:30, Z:5}
- DocB {X:100, Y:20}
- DocC {M:5, N:20, Z:5}

### (1) build stripes for all the test data sets - run the commands and insure that your output matches the output below

In [232]:
###########################################################################
# Make Stripes from ngrams for systems test 1
###########################################################################

!hdfs dfs rm --recursive systems_test_stripes_1
!python buildStripes.py -r local googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt > systems_test_stripes_1

rm: Unknown command
Did you mean -rm?  This command begins with a dash.
No configs found; falling back on auto-configuration
Creating temp directory /tmp/buildStripes.root.20170211.235301.697009
Running step 1 of 1...
Streaming final output from /tmp/buildStripes.root.20170211.235301.697009/output...
Removing temp directory /tmp/buildStripes.root.20170211.235301.697009...


In [233]:
!cat systems_test_stripes_1

"a"	{"limited": 55, "female": 447, "general": 92, "sea": 62, "in": 1201, "religious": 59, "george": 92, "biography": 92, "city": 62, "for": 59, "tales": 123, "child's": 1099, "forms": 116, "wales": 1099, "christmas": 1099, "government": 102, "collection": 239, "by": 62, "case": 604, "circumstantial": 62, "fairy": 123, "of": 1011, "study": 604, "bill": 59, "establishing": 59, "narrative": 62, "the": 124}
"bill"	{"a": 59, "religious": 59, "for": 59, "establishing": 59}
"biography"	{"a": 92, "of": 92, "george": 92, "general": 92}
"by"	{"a": 62, "city": 62, "the": 62, "sea": 62}
"case"	{"a": 604, "limited": 55, "government": 102, "of": 502, "study": 604, "female": 447, "in": 102}
"child's"	{"a": 1099, "wales": 1099, "christmas": 1099, "in": 1099}
"christmas"	{"a": 1099, "wales": 1099, "in": 1099, "child's": 1099}
"circumstantial"	{"a": 62, "of": 62, "the": 62, "narrative": 62}
"city"	{"a": 62, "the": 62, "by": 62, "sea": 62}
"collection"	{"a": 239, "forms": 116, "fairy": 123, "tales": 123,

<pre>
"a"	{"limited": 55, "sea": 62, "general": 92, "female": 447, "in": 1201, "religious": 59, "george": 92, "biography": 92, "city": 62, "for": 59, "tales": 123, "child's": 1099, "forms": 116, "wales": 1099, "christmas": 1099, "government": 102, "collection": 239, "by": 62, "case": 604, "circumstantial": 62, "fairy": 123, "of": 1011, "study": 604, "bill": 59, "establishing": 59, "narrative": 62, "the": 124}
"bill"	{"a": 59, "religious": 59, "for": 59, "establishing": 59}
"biography"	{"a": 92, "of": 92, "george": 92, "general": 92}
"by"	{"a": 62, "city": 62, "the": 62, "sea": 62}
"case"	{"a": 604, "limited": 55, "government": 102, "of": 502, "study": 604, "female": 447, "in": 102}
"child's"	{"a": 1099, "wales": 1099, "christmas": 1099, "in": 1099}
"christmas"	{"a": 1099, "wales": 1099, "in": 1099, "child's": 1099}
"circumstantial"	{"a": 62, "of": 62, "the": 62, "narrative": 62}
"city"	{"a": 62, "the": 62, "by": 62, "sea": 62}
"collection"	{"a": 239, "of": 355, "fairy": 123, "tales": 123, "forms": 116}
"establishing"	{"a": 59, "bill": 59, "religious": 59, "for": 59}
"fairy"	{"a": 123, "of": 123, "tales": 123, "collection": 123}
"female"	{"a": 447, "case": 447, "study": 447, "of": 447}
"for"	{"a": 59, "bill": 59, "religious": 59, "establishing": 59}
"forms"	{"a": 116, "of": 232, "collection": 116}
"general"	{"a": 92, "of": 92, "george": 92, "biography": 92}
"george"	{"a": 92, "of": 92, "biography": 92, "general": 92}
"government"	{"a": 102, "case": 102, "study": 102, "in": 102}
"in"	{"a": 1201, "case": 102, "government": 102, "study": 102, "child's": 1099, "wales": 1099, "christmas": 1099}
"limited"	{"a": 55, "case": 55, "study": 55, "of": 55}
"narrative"	{"a": 62, "of": 62, "the": 62, "circumstantial": 62}
"of"	{"a": 1127, "case": 502, "circumstantial": 62, "george": 92, "limited": 55, "tales": 123, "collection": 471, "general": 92, "forms": 348, "female": 447, "narrative": 62, "study": 502, "fairy": 123, "the": 62, "biography": 92}
"religious"	{"a": 59, "bill": 59, "for": 59, "establishing": 59}
"sea"	{"a": 62, "city": 62, "the": 62, "by": 62}
"study"	{"a": 604, "case": 604, "limited": 55, "government": 102, "of": 502, "female": 447, "in": 102}
"tales"	{"a": 123, "of": 123, "fairy": 123, "collection": 123}
"the"	{"a": 124, "city": 62, "circumstantial": 62, "of": 62, "sea": 62, "narrative": 62, "by": 62}
"wales"	{"a": 1099, "in": 1099, "christmas": 1099, "child's": 1099}

</pre>

In [234]:
###########################################################################
# Make Stripes from ngrams for systems test 2
###########################################################################

!hdfs dfs rm --recursive systems_test_stripes_2
!python buildStripes.py -r local atlas-boon-systems-test.txt > systems_test_stripes_2

rm: Unknown command
Did you mean -rm?  This command begins with a dash.
No configs found; falling back on auto-configuration
Creating temp directory /tmp/buildStripes.root.20170211.235636.185095
Running step 1 of 1...
Streaming final output from /tmp/buildStripes.root.20170211.235636.185095/output...
Removing temp directory /tmp/buildStripes.root.20170211.235636.185095...


In [235]:
!cat systems_test_stripes_2

"atlas"	{"dipped": 15, "boon": 50}
"boon"	{"atlas": 50, "dipped": 10, "cava": 10}
"cava"	{"dipped": 10, "boon": 10}
"dipped"	{"atlas": 15, "boon": 10, "cava": 10}


<pre>
"atlas"   {"dipped": 15, "boon": 50}   
"boon"    {"atlas": 50, "dipped": 10, "cava": 10}   
"cava"    {"dipped": 10, "boon": 10} 
"dipped"  {"atlas": 15, "boon": 10, "cava": 10}
</pre>

In [117]:
########################################################################
# Stripes for systems test 3 (given, no need to build stripes)
########################################################################

with open("systems_test_stripes_3", "w") as f:
    f.writelines([
        '"DocA"\t{"X":20, "Y":30, "Z":5}\n',
        '"DocB"\t{"X":100, "Y":20}\n',  
        '"DocC"\t{"M":5, "N":20, "Z":5, "Y":1}\n'
    ])
!cat systems_test_stripes_3   

"DocA"	{"X":20, "Y":30, "Z":5}
"DocB"	{"X":100, "Y":20}
"DocC"	{"M":5, "N":20, "Z":5, "Y":1}


### (2) Build Inverted Index - run the commands and insure that your output matches the output below

In [236]:
!python invertedIndex.py -r local systems_test_stripes_1 > systems_test_index_1

No configs found; falling back on auto-configuration
Creating temp directory /tmp/invertedIndex.root.20170211.235658.196230
Running step 1 of 1...
Streaming final output from /tmp/invertedIndex.root.20170211.235658.196230/output...
Removing temp directory /tmp/invertedIndex.root.20170211.235658.196230...


In [237]:
!python invertedIndex.py -r local systems_test_stripes_2 > systems_test_index_2

No configs found; falling back on auto-configuration
Creating temp directory /tmp/invertedIndex.root.20170211.235707.378783
Running step 1 of 1...
Streaming final output from /tmp/invertedIndex.root.20170211.235707.378783/output...
Removing temp directory /tmp/invertedIndex.root.20170211.235707.378783...


In [238]:
!python invertedIndex.py -r local systems_test_stripes_3 > systems_test_index_3

No configs found; falling back on auto-configuration
Creating temp directory /tmp/invertedIndex.root.20170211.235711.141870
Running step 1 of 1...
Streaming final output from /tmp/invertedIndex.root.20170211.235711.141870/output...
Removing temp directory /tmp/invertedIndex.root.20170211.235711.141870...


In [239]:
##########################################################
# Pretty print systems tests for generating Inverted Index
##########################################################

import json

for i in range(1,4):
  print "—"*100
  print "Systems test ",i," - Inverted Index"
  print "—"*100  
  with open("systems_test_index_"+str(i),"r") as f:
      lines = f.readlines()
      for line in lines:
          line = line.strip()
          word,stripe = line.split("\t")
          stripe = json.loads(stripe)
          stripe.extend([["",""] for _ in xrange(3 - len(stripe))])

          print "{0:>16} |{1:>16} |{2:>16} |{3:>16}".format(
              (word), stripe[0][0]+" "+str(stripe[0][1]), stripe[1][0]+" "+str(stripe[1][1]), stripe[2][0]+" "+str(stripe[2][1]))
        


————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  1  - Inverted Index
————————————————————————————————————————————————————————————————————————————————————————————————————
             "a" |          bill 4 |     biography 4 |            by 4
          "bill" |            a 27 |  establishing 4 |           for 4
     "biography" |            a 27 |       general 4 |        george 4
            "by" |            a 27 |          city 4 |           sea 4
          "case" |            a 27 |        female 4 |    government 4
       "child's" |            a 27 |     christmas 4 |            in 7
     "christmas" |            a 27 |       child's 4 |            in 7
"circumstantial" |            a 27 |     narrative 4 |           of 15
          "city" |            a 27 |            by 4 |           sea 4
    "collection" |            a 27 |         fairy 4 |         forms 3
  "establishing" |            a 27 |          bill 4 |

### Inverted Index

In [ ]:
————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  1  - Inverted Index
————————————————————————————————————————————————————————————————————————————————————————————————————
             "a" |          bill 4 |     biography 4 |            by 4
          "bill" |            a 27 |  establishing 4 |           for 4
     "biography" |            a 27 |       general 4 |        george 4
            "by" |            a 27 |          city 4 |           sea 4
          "case" |            a 27 |        female 4 |    government 4
       "child's" |            a 27 |     christmas 4 |            in 7
     "christmas" |            a 27 |       child's 4 |            in 7
"circumstantial" |            a 27 |     narrative 4 |           of 15
          "city" |            a 27 |            by 4 |           sea 4
    "collection" |            a 27 |         fairy 4 |         forms 3
  "establishing" |            a 27 |          bill 4 |           for 4
         "fairy" |            a 27 |    collection 5 |           of 15
        "female" |            a 27 |          case 7 |           of 15
           "for" |            a 27 |          bill 4 |  establishing 4
         "forms" |            a 27 |    collection 5 |           of 15
       "general" |            a 27 |     biography 4 |        george 4
        "george" |            a 27 |     biography 4 |       general 4
    "government" |            a 27 |          case 7 |            in 7
            "in" |            a 27 |          case 7 |       child's 4
       "limited" |            a 27 |          case 7 |           of 15
     "narrative" |            a 27 |circumstantial 4 |           of 15
            "of" |            a 27 |     biography 4 |          case 7
     "religious" |            a 27 |          bill 4 |  establishing 4
           "sea" |            a 27 |            by 4 |          city 4
         "study" |            a 27 |          case 7 |        female 4
         "tales" |            a 27 |    collection 5 |         fairy 4
           "the" |            a 27 |            by 4 |circumstantial 4
         "wales" |            a 27 |       child's 4 |     christmas 4
————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  2  - Inverted Index
————————————————————————————————————————————————————————————————————————————————————————————————————
         "atlas" |          boon 3 |        dipped 3 |                
          "boon" |         atlas 2 |          cava 2 |        dipped 3
          "cava" |          boon 3 |        dipped 3 |                
        "dipped" |         atlas 2 |          boon 3 |          cava 2
————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  3  - Inverted Index
————————————————————————————————————————————————————————————————————————————————————————————————————
             "M" |          DocC 4 |                 |                
             "N" |          DocC 4 |                 |                
             "X" |          DocA 3 |          DocB 2 |                
             "Y" |          DocA 3 |          DocB 2 |          DocC 4
             "Z" |          DocA 3 |          DocC 4 |                


### (3) Calculate similarities - run the commands and insure that your output matches the output below

#### NOTE: you must run in hadoop mode to generate sorted similarities

In [248]:
!python similarity.py -r hadoop systems_test_index_1 > systems_test_similarities_1

No configs found; falling back on auto-configuration
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/similarity.root.20170212.012901.423879
Copying local files to hdfs:///user/root/tmp/mrjob/similarity.root.20170212.012901.423879/files/...
Running step 1 of 2...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob8660531504806535883.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Total input paths to process : 1
  number of splits:2
  Submitting tokens for job: job_1485626579957_0221
  Submitted application application_1485626579957_0221
  The url to track the job: http://quickstart.cloudera:8088

In [251]:
!python similarity.py -r hadoop systems_test_index_2 > systems_test_similarities_2

No configs found; falling back on auto-configuration
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/similarity.root.20170212.014634.928830
Copying local files to hdfs:///user/root/tmp/mrjob/similarity.root.20170212.014634.928830/files/...
Running step 1 of 2...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob6612064399317853092.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Total input paths to process : 1
  number of splits:2
  Submitting tokens for job: job_1485626579957_0225
  Submitted application application_1485626579957_0225
  The url to track the job: http://quickstart.cloudera:8088

In [250]:
!python similarity.py -r hadoop systems_test_index_3 > systems_test_similarities_3

No configs found; falling back on auto-configuration
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/similarity.root.20170212.014448.786003
Copying local files to hdfs:///user/root/tmp/mrjob/similarity.root.20170212.014448.786003/files/...
Running step 1 of 2...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.8.0.jar] /tmp/streamjob8063775513243738256.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Total input paths to process : 1
  number of splits:2
  Submitting tokens for job: job_1485626579957_0223
  Submitted application application_1485626579957_0223
  The url to track the job: http://quickstart.cloudera:8088

In [255]:
############################################
# Pretty print systems tests FOR ALL TESTS - NOT USED - SEE BELOW
############################################

import json
for i in range(1,4):
  print '—'*110
  print "Systems test ",i," - Similarity measures"
  print '—'*110
  print "{0:>15} |{1:>27} |{2:>15} |{3:>15} |{4:>15} |{5:>15}".format(
          "average", "pair", "cosine", "jaccard", "overlap", "dice")
  print '-'*110

  with open("systems_test_similarities_"+str(i),"r") as f:
      lines = f.readlines()
      for line in lines:
          line = line.strip()
          avg,stripe = line.split("\t")
          stripe = json.loads(stripe)

          print "{0:>15f} |{1:>27} |{2:>15f} |{3:>15f} |{4:>15f} |{5:>15f}".format(
              float(avg), stripe[0], float(stripe[1]), float(stripe[2]), float(stripe[3]), float(stripe[4]))
 

——————————————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  1  - Similarity measures
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
        average |                       pair |         cosine |        jaccard |        overlap |           dice
--------------------------------------------------------------------------------------------------------------
       1.000000 |           female - limited |       1.000000 |       1.000000 |       1.000000 |       1.000000
       0.868292 |              forms - tales |       0.866025 |       0.750000 |       1.000000 |       0.857143
       0.868292 |              fairy - forms |       0.866025 |       0.750000 |       1.000000 |       0.857143
       0.830357 |               case - study |       0.857143 |       0.750000 |       0.857143 |       0.857143
       0.712500 |            child's - wales |       0.750000 |

### Pairwise Similairity 

In [ ]:
Systems test  3  - Similarity measures
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
        average |           pair |         cosine |        jaccard |        overlap |           dice
--------------------------------------------------------------------------------------------------------------
       0.741582 |    DocA - DocB |       0.816497 |       0.666667 |       1.000000 |       0.800000
       0.488675 |    DocA - DocC |       0.577350 |       0.400000 |       0.666667 |       0.571429
       0.276777 |    DocB - DocC |       0.353553 |       0.200000 |       0.500000 |       0.333333
--------------------------------------------------------------------------------------------------------------

Systems test  2  - Similarity measures
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
        average |           pair |         cosine |        jaccard |        overlap |           dice
--------------------------------------------------------------------------------------------------------------
       1.000000 |   atlas - cava |       1.000000 |       1.000000 |       1.000000 |       1.000000
       0.625000 |  boon - dipped |       0.666667 |       0.500000 |       0.666667 |       0.666667
       0.389562 |  cava - dipped |       0.408248 |       0.250000 |       0.500000 |       0.400000
       0.389562 |    boon - cava |       0.408248 |       0.250000 |       0.500000 |       0.400000
       0.389562 | atlas - dipped |       0.408248 |       0.250000 |       0.500000 |       0.400000
       0.389562 |   atlas - boon |       0.408248 |       0.250000 |       0.500000 |       0.400000
--------------------------------------------------------------------------------------------------------------

Systems test  1  - Similarity measures
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
        average |           pair |         cosine |        jaccard |        overlap |           dice
--------------------------------------------------------------------------------------------------------------
       0.096639 |      bill - of |       0.129099 |       0.055556 |       0.250000 |       0.105263
       0.096639 |   child's - of |       0.129099 |       0.055556 |       0.250000 |       0.105263
       0.096639 | christmas - of |       0.129099 |       0.055556 |       0.250000 |       0.105263
       0.096639 |establishing - of |       0.129099 |       0.055556 |       0.250000 |       0.105263
       0.096639 |       for - of |       0.129099 |       0.055556 |       0.250000 |       0.105263
       0.096639 | of - religious |       0.129099 |       0.055556 |       0.250000 |       0.105263
       0.096639 |     of - wales |       0.129099 |       0.055556 |       0.250000 |       0.105263
       0.120879 |       in - the |       0.142857 |       0.076923 |       0.142857 |       0.142857
       0.142202 |collection - in |       0.169031 |       0.090909 |       0.200000 |       0.166667
       0.142328 |      a - forms |       0.222222 |       0.071429 |       0.666667 |       0.133333
       0.156933 |    bill - case |       0.188982 |       0.100000 |       0.250000 |       0.181818
      ...

# === END OF PHASE 1 ===